<a href="https://colab.research.google.com/github/eeseohyun/project/blob/main/AI_05_%EC%9D%B4%EC%84%9C%ED%98%84_CP1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**라이브러리 불러오기**

In [ ]:
import os
import cv2
import numpy as np
import random
import keras

from imutils import paths
import matplotlib.pyplot as plt
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout,BatchNormalization,MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report

**With_Mask**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
images_withmask = os.listdir('/content/drive/MyDrive/Colab Notebooks/New Masks Dataset/Train/Mask')
sample_img_withmask = random.choice(images_withmask)
image = load_img("/content/drive/MyDrive/Colab Notebooks/New Masks Dataset/Train/Mask/"+sample_img_withmask)
plt.imshow(image)

**Without_Mask**

In [ ]:
images_withoutmask = os.listdir('/content/drive/MyDrive/Colab Notebooks/New Masks Dataset/Train/Non Mask')
sample_img_withoutmask = random.choice(images_withoutmask)
image = load_img("/content/drive/MyDrive/Colab Notebooks/New Masks Dataset/Train/Non Mask/"+sample_img_withoutmask)
plt.imshow(image)

**Data Augmentation (데이터 증식)**

In [ ]:
# 너비&높이 정의
img_width = 224
img_height = 224

# 초기 학습률, 훈련할 Epoch 수 및 배치 크기 초기화
LearningRate = 1e-4
EPOCHS = 50
BatchSize = 32
train_dir = "/content/drive/MyDrive/Colab Notebooks/New Masks Dataset/Train"
test_dir = "/content/drive/MyDrive/Colab Notebooks/New Masks Dataset/Validation"

In [ ]:
all_train_imagepaths = list(paths.list_images(train_dir))
all_test_imagepaths = list(paths.list_images(test_dir))
train_data = []
train_labels = []
test_data = []
test_labels = []

In [ ]:
# train 이미지 경로에 대한 루프
for imagePath in all_train_imagepaths:
  label = imagePath.split(os.path.sep)[-2]     # 파일명에서 클래스 레이블 추출
  
  image = load_img(imagePath, target_size=(224,224))     # 입력 이미지(224x224)를 로드하고 전처리
  image = img_to_array(image)
  image = preprocess_input(image)    # 이미지 배치를 인코딩하는 텐서 또는 Numpy 배열을 전처리
  
  # 데이터 및 레이블을 각각 업데이트
  train_data.append(image)
  train_labels.append(label)

In [ ]:
# test 이미지 경로에 대한 루프
for imagePath in all_test_imagepaths:
	# extract the class label from the filename
	label = imagePath.split(os.path.sep)[-2]

	# load the input image (224x224) and preprocess it
	image = load_img(imagePath, target_size=(224, 224))

	image = img_to_array(image)
	image = preprocess_input(image)

	# update the data and labels lists, respectively
	test_data.append(image)
	test_labels.append(label)

In [ ]:
def convert_data_labels(data,labels):
  # 데이터와 레이블을 NumPy 배열로 변환
  data = np.array(data, dtype="float32")
  labels = np.array(labels)
  
  # 라벨 이진화
  lb = LabelBinarizer()
  labels = lb.fit_transform(labels)
  # 라벨 원핫 인코딩
  labels = to_categorical(labels)
  return  data,labels

In [ ]:
train_data,train_labels = convert_data_labels(train_data,train_labels)
test_data,test_labels = convert_data_labels(test_data,test_labels)

In [ ]:
print("Size of train dataset : ",train_data.shape[0])
print("Size of test dataset : ",test_data.shape[0])

In [ ]:
# ImageDataGenerator를 이용한 데이터 증식
train_datagen = ImageDataGenerator(
    rotation_range = 45,     # 회전에 대한 각도 범위
    zoom_range = 0.15,       # [아래, 위] 임의 확대/축소 범위
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.15,      # 반시계 방향의 전단 각도
    horizontal_flip = True,  # 무작위로 입력을 수평으로 뒤집기
    fill_mode = 'nearest')   #'constant': kkkkkkkk|abcd|kkkkkkkk(cval=k)
                             #'nearest': aaaaaaaa|abcd|dddddddd
                             #'reflect': abcddcba|abcd|dcbaabcd
                             #'warp': abcdabcd|abcd|abcdabcd

train_generator = train_datagen.flow(train_data, train_labels, batch_size=BatchSize)

**Modeling**

- MobileNetV2

In [ ]:
mobilenet = MobileNetV2(weights="imagenet", # ImageNet에 대한 사전 학습
                        include_top=False,  # 네트워크 상단에 완전 연결 계층x
                        input_tensor=Input(shape=(224, 224, 3))) # 3개의 입력 채널

In [ ]:
# 기본 모델을 기반으로 head모델 구성
headModel = mobilenet.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

In [ ]:
# head모델을 기본 모델 위에 배치(= 실제 훈련할 모델)
model = Model(inputs=mobilenet.input, outputs=headModel)

In [ ]:
# Compile Model
opt = Adam(learning_rate=LearningRate, decay=LearningRate / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

In [ ]:
# 첫 훈련과정 중 업데이트되지 않기 위해 기본 모델의 모든 레이어를 반복하고 고정
for layer in mobilenet.layers:
	layer.trainable = False

**Callback Function**

In [ ]:
# checkpoint = ModelCheckpoint('model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')
# # history=model.fit(train_data,train_target,epochs=100,callbacks=[checkpoint],validation_split=0.2)
# EarlyStopping
earlystop = EarlyStopping(monitor = 'val_loss',
                          min_delta = 0,
                          patience = 7,
                          verbose = 1,
                          restore_best_weights = True)

# ModelCheckPoint
checkPoint = keras.callbacks.ModelCheckpoint(filepath="/content/sample_data/fmd_model.h5",
                             monitor='val_loss',
                             mode='auto',
                             save_best_only=True,
                             verbose=1)

# ReduceLROnPlateau
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=2, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [ ]:
callbacks = [earlystop , checkPoint, learning_rate_reduction]

**Model Fitting**

In [ ]:
classifier = model.fit(
    train_datagen.flow(train_data, train_labels, batch_size=BatchSize), 
    epochs=EPOCHS,
    validation_data=(test_data,test_labels),
    validation_steps=len(test_data)//BatchSize,
    steps_per_epoch=len(train_data)//BatchSize,
    callbacks=callbacks
)

**Training Loss and Accuracy Visualization**

In [ ]:
plt.style.use('ggplot')
N = 10
plt.figure()
plt.plot( classifier.history["loss"], label='train_loss')
plt.plot( classifier.history['val_loss'], label='val_loss')
plt.plot( classifier.history['accuracy'], label='train_acc')
plt.plot( classifier.history['val_accuracy'], label='val_acc')
plt.title('Training Loss and Accuracy')
plt.xlabel("Epoch")
plt.ylabel("Loss/Accuracy")
plt.legend(loc='center right')

**Find Accuracy**

In [ ]:
val_loss,val_acc = model.evaluate(test_data,test_labels)

print("Accuracy: ",val_acc)
print("=======================================================")
print("Loss: ",val_loss)

##**라이브러리 불러오기**

In [ ]:
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import argparse
import imutils
import time

**알람 소리**

In [ ]:
!pip install pygame

In [ ]:
import pygame
pygame.init()
sound = pygame.mixer.Sound('/content/drive/MyDrive/Colab Notebooks/mixkit-sound-alert-in-hall-1006.wav')

**Image Pre-Processing**

In [ ]:
def mask_detection_prediction(frame, faceNet, maskNet):
  # 프레임 차원을 찾고 blob 구성
  (h, w) = frame.shape[:2]
  blob = cv2.dnn.blobFromImage(frame, 1.0, (224, 224),(104.0, 177.0, 123.0))

  # 네트워크를 통해 blob을 전달하고 얼굴 감지
  faceNet.setInput(blob)
  detections = faceNet.forward()

  # 얼굴, 얼굴 위치 및 예측 목록을 저장할 빈 목록 생성
  faces = []
  locs = []
  preds = []

  # loop
  for i in range(0, detections.shape[2]):
    # 탐지와 관련된 신뢰도 또는 확률 찾기
    confidence = detections[0, 0, i, 2]

    # 강력한 탐지 필터링
    if confidence > 0.5:

        # bounding box의 시작, 끝 좌표 찾기
        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype("int")

        # bounding box  가 프레임 크기 내에 있는지 확인
        (startX, startY) = (max(0, startX), max(0, startY))
        (endX, endY) = (min(w - 1, endX), min(h - 1, endY))

        # 얼굴 ROI 추출, BGR에서 RGB 채널로 변환
        # ordering, (224x224) 사이즈 조정, 전처리
        face = frame[startY:endY, startX:endX]
        face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
        face = cv2.resize(face, (224, 224))
        face = img_to_array(face)
        face = preprocess_input(face)

        # 얼굴 및 bounding box를 해당 목록에 추가
        faces.append(face)
        locs.append((startX, startY, endX, endY))

    # 적어도 하나의 얼굴이 감지된 경우에만 예측
    if len(faces) > 0:
        # 더 빠른 추론을 위해 위의 'for' 루프에서 하나씩 예측하는 대신 모든 얼굴에 대해 동시에 일괄 예측을 수행
        faces = np.array(faces, dtype="float32")
        preds = maskNet.predict(faces, batch_size=32)

    # 얼굴 위치와 해당 예측의 2튜플을 반환
    return (locs, preds)

##**Load Caffe Model**

- **Caffe**(Convolutional Architecture for Fast Feature Embedding)

  : 사용자가 이미지 분류 및 이미지 분할 모델을 생성할 수 있도록 하는 딥러닝 프레임워크



In [ ]:
from os.path import dirname, join

prototxtPath = join("face_detector", "deploy.prototxt")
weightsPath = join("face_detector", "res10_300x300_ssd_iter_140000.caffemodel")

faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

# 디스크에서 안면 마스크 감지기 모델 로드
maskNet = load_model("fmd_model.h5")

**Face Detection on Live Camera**

In [ ]:
# initialize the video stream
vs = VideoStream(src=0).start()

# VideoStream의 frame 반복
while True:
    
    # 스레드된 비디오 스트림에서 프레임을 잡고 너비가 400픽셀이 되도록 크기 조정
    frame = vs.read()
    frame = imutils.resize(frame, width=400)

    # 프레임에서 얼굴을 감지하고 마스크 착용여부 확인
    (locs, preds) = mask_detection_prediction(frame, faceNet, maskNet)

    # 감지된 얼굴 위치와 해당 위치에 대한 루프
    for (box, pred) in zip(locs, preds):
        # bounding box 및 예측 압축 풀기
        (startX, startY, endX, endY) = box
        (mask, withoutMask) = pred

        if mask>withoutMask:
            label = "Mask"
            color = (0, 255, 0)

            print("Normal")
        else:
            label = "No Mask"
            color = (0, 0, 255)
            sound.play()

            print("Alert!!!")

        # 레이블에 확률 포함
        label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)

        # 출력 프레임에 레이블 및 경계 상자에 사각형 표시
        cv2.putText(frame, label, (startX, startY - 10),cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)

    # 출력 프레임 표시
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF

    # `q` 키가 눌렸다면 루프 정지
    if key == ord("q"):
        break


cv2.destroyAllWindows()
vs.stop()